In [110]:
import pandas as pd
from datetime import datetime

In [111]:
# Задание 1

# Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей,
# которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением
# столбца timestamp для данного значения userId.

# Создаем функцию, возвращающую период между датой первой оценки и датой второй оценки в формате datetime 
def get_time(row):
    return datetime.fromtimestamp(row['max'])-datetime.fromtimestamp(row['min'])

# Читаем csv файл в датафрейм
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# Группируем по полю userId, записываем даты первой (min) и последней (max) оценок пользователя,
# а также количество оценок
ratings = ratings.groupby('userId').agg({'timestamp': ['min', 'max'], 'movieId': 'count'}).reset_index()

# Переименовываем столбцы для дальнейшего использования
ratings.columns = ['userId','min','max','movies']

# Фильтруем по полю movies - отбираем пользователей, поставивших 100 оценок и более
ratings = ratings[(ratings['movies'] >= 100.0)]

# С помощью подготовленной функции для каждой строки считаем разницу между датами первой и последней оценки
ratings['lifetime'] = ratings.apply(get_time, axis=1)

# Выводим среднее время жизни пользователей по всем записям
print(ratings[['lifetime']].mean())

lifetime   455 days 05:29:08.566539
dtype: timedelta64[ns]


In [112]:
# Задание 2

# Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две
# таблицы: таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
# аналогичную таблицу по типам выручки с указанием адреса клиента
# Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)


auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

client_base.set_index('client_id', inplace=True)
rzd.set_index('client_id', inplace=True)
auto.set_index('client_id', inplace=True)
air.set_index('client_id', inplace=True)

# Вариант 1 - объединение на основе справочника клиентов (включая адрес клиента)
joined_one = client_base.join(rzd,how='left')
joined_one = joined_one.join(auto,how='left')
joined_one = joined_one.join(air,how='left')
joined_one.fillna(0,inplace=True)
joined_one

,address,rzd_revenue,auto_revenue,air_revenue
client_id,,,,
111,Комсомольская 4,1093.0,0.0,0.0
112,Энтузиастов 8а,2810.0,0.0,0.0
113,Левобережная 1а,10283.0,57483.0,0.0
114,Мира 14,5774.0,83.0,0.0
115,ЗЖБИиДК 1,981.0,912.0,81.0
116,Строителей 18,0.0,4834.0,4.0
117,Панфиловская 33,0.0,98.0,13.0
118,Мастеркова 4,0.0,0.0,173.0


In [113]:
# Вариант 2 - объединение на основе отдельных таблиц (без адреса клиента)
joined_two = rzd.join(auto,how='outer')
joined_two = joined_two.join(air, how='outer')
joined_two.fillna(0,inplace=True)
joined_two

,rzd_revenue,auto_revenue,air_revenue
client_id,,,
111,1093.0,0.0,0.0
112,2810.0,0.0,0.0
113,10283.0,57483.0,0.0
114,5774.0,83.0,0.0
115,981.0,912.0,81.0
116,0.0,4834.0,4.0
117,0.0,98.0,13.0
118,0.0,0.0,173.0


In [ ]:
# Задание 3
# В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id
# известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы.
# Как бы вы добавили эти сведения в таблицу визитов и покупок?

К сожалению не совсем понятно, что значит "последовательность координат" и "когда они требовались приложению для работы".
Предполагаю, что есть таблица, в которой имеются строки с неуникальными значениями user_id и соответствующим им двумя
координатами, а также возможно другими полями.

Также не вполне понимаю, что значит "добавить эти сведения в таблицу визитов и покупок". Если вопрос в проектировании
структуры, в которую затем будут автоматически добавляться данные - есть два варианта:
    1) Добавлять сами координаты в таблицу визитов и в таблицу покупок, а не записывать их в отдельную таблицу. В целом
    это выглядит предпочтительней (создание отдельной таблицы для хранения всего двух полей, к которым необходимо
    будет добавить идентификатор - id пользователя или id визита/покупки, кажется избыточным)
        
    2) Если же таблица с userid и координатами - это некая данность, то в нее необходимо добавить поле идентификатора
    (например, actionId), в котором будет храниться либо id визита, либо id покупки (если они не могут совпадать),
    или два поля visitId и orderId, по которым можно привязать координаты к тому или иному событию (визит/покупка). 


# Для составления ответа можно использовать вопросы:
# 1. У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего
# потребуется одно-два числа. Как их получить?
Опять же не совсем понимаю, что значит "как их получить"? В идеале при работе системы координаты привязываются
к визиту/покупке автоматически. Если же это не наш случай, и нужно сопоставить визиты/покупки c координатами, то
вероятно, это можно сделать это по userId и данным по дате/времени (если эти данные имеются и в таблице визитов/покупок
и в таблице с координатами). 

# 2. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга.
# Как это отразится на вопросе расчетах пункта 1?
Не понял вопрос

# 3. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что
# дают.
По координатам можно получить название местонахождения (обратное геокодирование). Оно можент включать следующие
параметры (в зависимости от сервиса):
    - Страна
    - Регион
    - Город
    - Улица
    - Дом
    - Индекс и т.п.


In [13]:
a = -6
if a >= 0:
    print('a')
elif a < -5:
    print('b')

b


In [20]:
example = 'abba'
print(example.replace('b','a'))
example

aaaa


'abba'

In [21]:
print(f'{123.444:.2f}')

123.44


In [23]:
f = open('movies_example.txt', 'r')
print(f)

<_io.TextIOWrapper name='movies_example.txt' mode='r' encoding='cp1252'>
